# -1) SAVE / RESUME / LOAD

In [ ]:
SAVE = True # NAME = ?
#---
RESUME = True # TIMESTAMP = ?
TIMESTAMP = ""
#---
LOAD_now = False
LOAD_past = True
LOAD = LOAD_now or LOAD_past

# 0) Drive mount & Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# cd ""

In [ ]:
!pwd
!ls -l | grep ^- | wc -l

In [ ]:
import os
from pathlib import Path
from glob import glob
import time
import datetime
from pytz import timezone
import sys
import json

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
# import cv2
import PIL
from sklearn.model_selection import train_test_split

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.optim as optim
#---
import torchvision
import torchvision.datasets as dsets
import torchvision.transforms as transforms
#---
from tqdm import tqdm # , trange

# 1) Environment setting

In [ ]:
current_dir = os.getcwd(); print(current_dir)
raw_data_dir = current_dir + "/raw_data"
result_dir = current_dir + "/result"
#for loc in [result_dir]: 
#     if not os.path.exists(loc): os.makedirs(loc)

In [ ]:
# True: GPU is avaliable / False: GPU is unavailable
USE_CUDA = torch.cuda.is_available()
# if USE_CUDA is TRUE: use GPU / else: use CPU
device = torch.device("cuda" if USE_CUDA else "cpu"); print(device)

In [ ]:
print("Python:", sys.version)
print("PyTorch:", torch.__version__)

In [ ]:
pd.set_option("max_colwidth", 100)
# pd.reset_option("max_colwidth")

# 2) Images

## Overview

In [ ]:
col_names = ["BMP", "Alt", "Diff", "Type", "Num", "Gender", "Hand", "Finger"]
data_category = {"All"       : {"color": "RGB", "addr_tail": "/SOCOFING"},
                 "Alt"       : {"color": "grayscale", "addr_tail": "/SOCOFING/Altered"},
                 "Real_rgb"  : {"color": "RGB", "addr_tail": "/SOCOFING/Real"},
                 "Real_gray" : {"color": "grayscale", "addr_tail": "/SOCOFING/Real"}
}

Real

In [ ]:
##### Real #####
start_time = time.time()
#---
Real_data_Path = Path("./raw_data/SOCOFING/Real/Real")
Real_BMP_Path = sorted(Real_data_Path.glob("*.BMP")) # r"*/*.BMP"
print(len(Real_BMP_Path))
#---
Real_Series = { name: [] for name in col_names}
#---
Real_Series["BMP"] = pd.Series(Real_BMP_Path, name = "BMP").astype(str)
#---
Real_name = list(map(lambda x: os.path.split(x)[1], Real_BMP_Path))
Real_Series["Num"] = pd.Series(list(map(lambda x: x.split("__")[0], Real_name)), name = "Num")
Real_info = list(map(lambda x: x.split("__")[1].split("_"), Real_name))
Real_Series["Gender"] = pd.Series(list(map(lambda x: x[0], Real_info)), name = "Gender")
Real_Series["Hand"] = pd.Series(list(map(lambda x: x[1], Real_info)), name = "Hand")
Real_Series["Finger"] = pd.Series(list(map(lambda x: x[2], Real_info)), name = "Finger")
#---
Real_Series["Alt"] = pd.Series(["No" for i in range(len(Real_BMP_Path))], name = "Alt")
Real_Series["Diff"] = pd.Series(["No" for i in range(len(Real_BMP_Path))], name = "Diff")
Real_Series["Type"] = pd.Series(["No" for i in range(len(Real_BMP_Path))], name = "Type")
#---
concat_list = [Real_Series[col] for col in col_names]
Real_Data = pd.concat(concat_list, axis=1)
print(time.time()-start_time, "sec")

In [ ]:
print(Real_Data.isnull().values.any()) # print(Real_Data.isna().sum())
Real_Data

In [ ]:
# 600 users / 1 user = 10 fingers
count = Real_Data.groupby('Num')['Num'].count()
# print(count)
print(len(count))
for i in count:
    if i == 10: pass
    else: print(i)

Altered

In [ ]:
##### Altered #####
start_time = time.time()
#---
Alt_data_Path = Path("./raw_data/SOCOFING/Altered")
Alt_BMP_Path = sorted(Alt_data_Path.glob(r"*/*.BMP")) # r"*/*.BMP"
print(len(Alt_BMP_Path))
#---
Alt_Series = { name: [] for name in col_names}
#---
Alt_Series["BMP"] = pd.Series(Alt_BMP_Path, name = "BMP").astype(str)
#---
Alt_name = list(map(lambda x: os.path.split(x)[1], Alt_BMP_Path))
Alt_Series["Num"] = pd.Series(list(map(lambda x: x.split("__")[0], Alt_name)), name = "Num")
Alt_info = list(map(lambda x: x.split("__")[1].split("_"), Alt_name))
Alt_Series["Gender"] = pd.Series(list(map(lambda x: x[0], Alt_info)), name = "Gender")
Alt_Series["Hand"] = pd.Series(list(map(lambda x: x[1], Alt_info)), name = "Hand")
Alt_Series["Finger"] = pd.Series(list(map(lambda x: x[2], Alt_info)), name = "Finger")
#---
Diff_Labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1].split("-")[1], Alt_BMP_Path))
for difficulty in ["Easy", "Medium", "Hard"]:
    print("Altered-" + difficulty, ":", Diff_Labels.count(difficulty))
#---
Alt_Series["Alt"] = pd.Series(["Yes" for i in range(len(Alt_BMP_Path))], name = "Alt")
Alt_Series["Diff"] = pd.Series(Diff_Labels, name = "Diff")
Alt_Series["Type"] = pd.Series(list(map(lambda x: x[4].split(".")[0], Alt_info)), name = "Type")
#---
concat_list = [Alt_Series[col] for col in col_names]
Alt_Data = pd.concat(concat_list, axis=1)
print(time.time()-start_time, "sec")

In [ ]:
print(Alt_Data.isnull().values.any()) # print(Alt_Data.isna().sum())
Alt_Data

In [ ]:
count = Alt_Data.groupby(['Num', 'Diff'])['Num'].count()
print(len(count))
print(count)

## Dataset (using torchvision)

In [ ]:
batch_size = 128
# num_workers = 1 # number of CPU (or GPU)
# n_iters = 1000
# num_epochs = n_iters / (len(features_train) / batch_size)
# num_epochs = int(num_epochs)

In [ ]:
trans = {"RGB": 0, "grayscale": 0}
pixel = 128
trans["RGB"] = transforms.Compose([#transforms.Grayscale(num_output_channels=1),
                                   transforms.Resize((pixel, pixel)),
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                   ])
trans["grayscale"] = transforms.Compose([transforms.Grayscale(num_output_channels=1),
                                   transforms.Resize((pixel, pixel)),
                                   transforms.ToTensor(),
                                   transforms.Normalize(0.5, 0.5)
                                   ])

In [ ]:
LABELS_UNIQUE = []
for person in list(map(lambda x: str(x), range(1, 600 + 1))):
    for hand in ["Right", "Left"]:
        for finger in ["thumb", "index", "middle", "ring", "little"]:
            LABELS_UNIQUE.append("_".join([person, hand, finger]))
print(LABELS_UNIQUE[:10])

In [ ]:
class SOCOFING_Dataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transforms=None, label=None, color="RGB", labels_unique = LABELS_UNIQUE):
        self.root_dir = root_dir
        self.classes = os.listdir(self.root_dir)
        self.transforms = transforms
        self.color = color
        #---
        self.class_data = {cls: [] for cls in self.classes}
        self.col_names = ["BMP", "Alt", "Diff", "Type", "Num", "Gender", "Hand", "Finger"]
        self.class_Series = {cls : {col: [] for col in self.col_names} for cls in self.classes}
        self.class_df = {cls: 0 for cls in self.classes}
        #---
        self.label_name = label
        self.labels = []
        self.labels_unique = labels_unique
        self.data = []
        self.DataFrame = 0
        
        for idx, cls in enumerate(self.classes):
            cls_dir = os.path.join(self.root_dir, cls)
            self.class_data[cls] = glob(os.path.join(cls_dir, '*.BMP')) # list
            # print(cls, len(self.class_data[cls]))
            if len(self.class_data[cls]) == 0:
                self.class_data[cls] = glob(os.path.join(cls_dir, r'*/*.BMP'))        
            #---
            ADDR = list(map(lambda x: x.split("/")[-3:], self.class_data[cls]))
            self.class_Series[cls]["BMP"] = pd.Series(list(map(lambda x: "/".join(x), ADDR)), name = "BMP")
            self.class_Series[cls]["Alt"] = pd.Series(list(map(lambda x: x[0], ADDR)), name = "Alt")
            self.class_Series[cls]["Diff"] = pd.Series(list(map(lambda x: x[1], ADDR)), name = "Diff")
            self.class_Series[cls]["Diff"] = self.class_Series[cls]["Diff"].replace({"Altered-Easy": "Easy", "Altered-Hard": "Hard", "Altered-Medium": "Medium"}) # Real: Real / Altered: Easy, Hard, Medium
            #---
            NAME = list(map(lambda x: x[2], ADDR))
            self.class_Series[cls]["Num"] = pd.Series(list(map(lambda x: x.split("__")[0], NAME)), name = "Num")
            #---
            NAME= list(map(lambda x: x.split("__")[1].split("_"), NAME))
            self.class_Series[cls]["Gender"] = pd.Series(list(map(lambda x: x[0], NAME)), name = "Gender")
            self.class_Series[cls]["Hand"] = pd.Series(list(map(lambda x: x[1], NAME)), name = "Hand")
            self.class_Series[cls]["Finger"] = pd.Series(list(map(lambda x: x[2], NAME)), name = "Finger")
            self.class_Series[cls]["Type"] = pd.Series(list(map(lambda x: x[-1].split(".")[0], NAME)), name = "Type")
            self.class_Series[cls]["Type"] = self.class_Series[cls]["Type"].replace({"finger": "Real"}) # Real: Real / Altered: Obl, Zcut, C
            #---
            concat_list = [self.class_Series[cls][col] for col in col_names]
            self.class_df[cls] = pd.concat(concat_list, axis=1)
            #--
            if self.label_name == None:
                self.class_df[cls]['Label'] = self.class_df[cls]["Alt"]
            else:
                self.class_df[cls]['Label'] = self.class_df[cls][self.label_name].agg('_'.join, axis=1)
            # end of for loop
        #---
        for cls in self.classes:
            self.data = self.data + self.class_data[cls]
        #---
        concat_list = [self.class_df[cls] for cls in self.classes]
        self.DataFrame = pd.concat(concat_list, axis = 0,  ignore_index = True)
        #---
        self.labels = list(self.DataFrame['Label'])
        #self.labels_unique = list(set(self.labels))
        #start_time = time.time()
        self.labels = list(map(lambda x: self.labels_unique.index(x), self.labels ))
        #print("labels:", time.time()-start_time, "sec")

    def __getitem__(self, idx):
        img_path, label = self.data[idx], self.labels[idx]
        if self.color == "grayscale":
            img = PIL.Image.open(img_path).convert('L')
        else:
            img = PIL.Image.open(img_path).convert('RGB')
        if self.transforms:
            img = self.transforms(img)

        info_dict = {col: self.DataFrame[col][idx] for col in col_names}
        #output = {'image':img, 'label':label, 'info':info_dict}
        output = (img, label) #, info_dict)

        return output

    def __len__(self):
        return len(self.data)

In [ ]:
label_list = ["Num", "Hand", "Finger"]
FING_dsets = {}
for cat in data_category.keys(): 
    color = data_category[cat]["color"]
    tail = data_category[cat]["addr_tail"]
    root_dir = raw_data_dir + tail

    FING_dsets[cat] = SOCOFING_Dataset(root_dir = root_dir, transforms = trans[color], label = label_list, color = color, labels_unique = LABELS_UNIQUE)
    print(cat, len(FING_dsets[cat]), end=" / ")

check

In [ ]:
# check 1
FING_dsets["All"].DataFrame

In [ ]:
# check 2
FING_dsets["Alt"].__getitem__(0)

In [ ]:
# check 3
FING_dsets["Real_rgb"].__getitem__(11)

In [ ]:
# check 3
FING_dsets["Real_gray"].__getitem__(11)

## Visualize

In [ ]:
def imshow(img):
    img = img/2 + 0.5 #unnormalize
    np_img = img.numpy()
    plt.imshow(np.transpose(np_img, (1,2,0)))
    #print(np_img.shape)
    #print((np.transpose(np_img, (1,2,0))).shape)

def imshow_one(addr):
    plt.figure(figsize=(2,2))
    img_name = raw_data_dir +"/SOCOFING/" + addr
    img = PIL.Image.open(img_name).convert("L")
    arr = np.asarray(img)
    #print(arr.shape)
    plt.title(addr)
    plt.imshow(arr, cmap='gray', vmin=0, vmax=255)

In [ ]:
FING_dataloader = {}
for i in data_category:
    FING_dataloader[i] = DataLoader(FING_dsets[i], batch_size = batch_size, shuffle = False)

In [ ]:
TEMP = "Real_gray"
dataloader = FING_dataloader[TEMP]
dataiter = iter(dataloader)
outputs = dataiter.next() # output = {'image':img, 'label':label, 'info':info_dict}
images, labels = outputs[0], outputs[1]
print(labels, "\n---")
#---
print(images.shape)
imshow(torchvision.utils.make_grid(images, nrow=8))
print("---")
#---
imshow_one(FING_dsets[TEMP].DataFrame["BMP"][0])
imshow_one(FING_dsets[TEMP].DataFrame["BMP"][1])
#imshow_one(FING_dsets[TEMP].DataFrame["BMP"][2])
#imshow_one(FING_dsets[TEMP].DataFrame["BMP"][3])
# it seems there is no problem

# 3) Split into Train / Validation / Test

In [ ]:
##### seed #####
random.seed(1234)
np.random.seed(1234)
torch.manual_seed(1234)
if device == 'cuda':
    torch.cuda.manual_seed_all(1234)

In [ ]:
Train_Set = {}
Validation_Set = {}
Test_Set = {}
for cat in data_category:
    num = len(FING_dsets[cat])
    train_size = int(0.6 * num)
    validation_size = int(0.2 * num)
    test_size = num - train_size - validation_size
    random.seed(1234)
    np.random.seed(1234)
    torch.manual_seed(1234)
    if device == 'cuda':
        torch.cuda.manual_seed_all(1234)
    Train_Set[cat], Validation_Set[cat], Test_Set[cat] = torch.utils.data.random_split(FING_dsets[cat], [train_size, validation_size, test_size], generator=torch.Generator().manual_seed(1234))
    print(cat, train_size, validation_size, test_size)

In [ ]:
for cat in data_category:
    lbl = Train_Set[cat][0][1]
    print(cat, ":", lbl, "=", FING_dsets[cat].labels_unique[lbl], end =" / ")
    print("unique:", len(FING_dsets[cat].labels_unique))
# (Perhaps..) All : 4154 = 416_Right_little / Alt : 3893 = 390_Right_ring / Real_rgb : 1378 = 138_Left_ring / Real_gray : 1378 = 138_Left_ring

In [ ]:
Train_loader = {}
Validation_loader = {}
Test_loader = {}
for cat in data_category:
    Train_loader[cat] = DataLoader(Train_Set[cat], batch_size = batch_size, shuffle = False)
    Validation_loader[cat] = DataLoader(Validation_Set[cat], batch_size = batch_size, shuffle = False)
    Test_loader[cat] = DataLoader(Test_Set[cat], batch_size = batch_size, shuffle = False)

# 4) Models

In [ ]:
all_models = ["TypicalCNN"]
model_dict = { m: 0 for m in all_models}

## 4.1) TypicalCNN

In [ ]:
class TypicalCNN(nn.Module):
    def __init__(self):
        super(TypicalCNN, self).__init__()
        # 1 input image channel, 16 output channels, 3x3 square convolution kernel
        self.conv1 = nn.Conv2d(1,  16, kernel_size=3, stride=2, padding=1) 
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout2d(0.4)
        self.batchnorm1 = nn.BatchNorm2d(16)
        self.batchnorm2 = nn.BatchNorm2d(32)
        self.batchnorm3 = nn.BatchNorm2d(64)
        self.fc1 = nn.Linear(64 * 8 * 8, 2048)
        self.fc2 = nn.Linear(2048, 1024)
        self.fc3 = nn.Linear(1024, 6000)
        
    def forward(self, x): # 128
        check = False
        if check: print(0, x.shape)
        x = self.batchnorm1(F.relu(self.conv1(x))) # 64
        if check: print(1, x.shape)
        x = self.batchnorm2(F.relu(self.conv2(x))) # 32
        if check: print(2, x.shape)
        x = self.dropout(self.batchnorm2(self.pool(x))) # 16
        if check: print(3, x.shape)
        x = self.batchnorm3(self.pool(F.relu(self.conv3(x)))) # 8
        if check: print(4, x.shape)
        x = self.dropout(self.conv4(x)) # 8
        if check: print(5, x.shape)
        x = x.view(-1, 64 * 8 * 8) # Flatten layer
        if check: print(6, x.shape)
        x = self.dropout(self.fc1(x))
        if check: print(7, x.shape)
        x = self.dropout(self.fc2(x))
        if check: print(8, x.shape)
        x = F.log_softmax(self.fc3(x), dim = 1)
        if check: print(9, x.shape)
        return x

Set model 

In [ ]:
if device == "cpu":
    model = TypicalCNN() 
else:
    model =  TypicalCNN().to(device)
print(model)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

Choose data

In [ ]:
DATA_cat = "Alt"
n_epochs = 30
# print_every = 10

In [ ]:
train_loader = Train_loader[DATA_cat]
validation_loader = Validation_loader[DATA_cat]
test_loader = Test_loader[DATA_cat]

Training preparation

In [ ]:
valid_loss_min = np.Inf
val_loss = []
val_acc = []
train_loss = []
train_acc = []
total_step = len(train_loader)

Save or not

In [ ]:
##### SAVE or not #####
NAME = "model_TypicalCnn"
#---
now_utc = datetime.datetime.now(timezone("UTC")); now_kst = now_utc.astimezone(timezone("Asia/Seoul")); print(now_kst)
nowDatetime = now_kst.strftime('%Y_%m_%d_%Hh%Mm%Ss'); print(nowDatetime)
if SAVE == True:
    saving_dir = result_dir + "/classify/" + nowDatetime
    for loc in [saving_dir]: 
        if not os.path.exists(loc): os.makedirs(loc)
    TXT_PATH = saving_dir + "/model_updated_epoch.txt"
    f = open(TXT_PATH, 'w')
    f.write("#"+ nowDatetime +"\n-----\n")
    f.close()

Resume or not

In [ ]:
##### RESUME or new #####
DICT_PATH_now = saving_dir + "/Cell_info.json"
DICT_PATH_past = result_dir + "/classify/" + TIMESTAMP +"/Cell_info.json"
# define Cell_info (dictionary)
Cell_info = {"time": nowDatetime, "epoch_past": 0, "best val.acc": -1, "best epoch": -1, "best val.loss": -1}
if RESUME == True: # SAVE == True
    json_file = open(DICT_PATH_past, "r")
    Cell_info = json.load(json_file)
else:
    pass
print(Cell_info)

Training

In [ ]:
##### Training #####
# Resume in current session or Not
if RESUME == True:
    if os.path.exists(DICT_PATH_now):
        json_file = open(DICT_PATH_now, "r")
        Cell_info = json.load(json_file)
else:
    pass
print(Cell_info)
#--- 
time.sleep(0.3)
start_time = {"epoch": 0, "loader":0, "model": 0}
#---
for epoch in range(1 + Cell_info["epoch_past"], n_epochs + 1 + Cell_info["epoch_past"]):
    model.train()
    with tqdm(train_loader, unit="batch") as tepoch:
        running_loss = 0.0
        # scheduler.step(epoch)
        correct = 0
        total=0
        # print(f'Epoch {epoch}\n')
        for data_, target_ in tepoch:
            tepoch.set_description(f"Epoch {epoch}")
        # for batch_idx, (data_, target_) in enumerate(train_loader):
            if device != "cpu": data_, target_ = data_.to(device), target_.to(device) # on GPU
            #-- zero the parameter gradients
            optimizer.zero_grad()
            #-- forward + backward + optimize
            outputs = model(data_)
            loss = criterion(outputs, target_)
            loss.backward()
            optimizer.step()
            #-- print statistics
            _, pred = torch.max(outputs, dim=1)
            loss_now = loss.item()
            correct_now = (pred == target_).sum().item()
            accuracy_now = correct_now / batch_size
            #--
            running_loss += loss_now
            correct += correct_now
            total += target_.size(0) # same as += batch_size
            #--
            tepoch.set_postfix(loss = loss.item(), accuracy =100. * accuracy_now, position = epoch - 1) #; time.sleep(0.1)
            # if (batch_idx) % 20 == 0:
            #     print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' .format(epoch, n_epochs, batch_idx, total_step, loss.item()))
        ### end of for ... in tepoch ###

        train_acc.append(100 * correct / total)
        train_loss.append(running_loss/total_step)
        print(f'Epoch {epoch}: train loss = {np.mean(train_loss):.4f}, train acc = {(100 * correct / total):.4f}')

        # validation and saving
        time.sleep(0.5)
        with torch.no_grad():
            model.eval()
            with tqdm(validation_loader, unit="batch") as tepoch:
                batch_loss = 0
                total_t=0
                correct_t=0
                for data_t, target_t in tepoch:
                    tepoch.set_description(f"Epoch {epoch}")
                    if device != "cpu": data_t, target_t = data_t.to(device), target_t.to(device) # on GPU
                    outputs_t = model(data_t)
                    _, pred_t = torch.max(outputs_t, dim=1)
                    loss_t = criterion(outputs_t, target_t)
                    #---
                    loss_t_now = loss_t.item()
                    correct_t_now = (pred_t == target_t).sum().item()
                    accuracy_t_now = correct_t_now / batch_size
                    #---
                    batch_loss += loss_t_now
                    correct_t += torch.sum(pred_t==target_t).item()
                    total_t += target_t.size(0)
                    #---
                    tepoch.set_postfix(loss = loss_t_now, accuracy =100. * accuracy_t_now, position = epoch - 1) #; time.sleep(0.1)
                ### end of for ... in tepoch ###
                val_acc.append(100 * correct_t / total_t)
                val_loss.append(batch_loss/len(validation_loader))
                network_learned = batch_loss < valid_loss_min
                print(f'Epoch {epoch}: val. loss = {np.mean(val_loss):.4f}, val. acc = {(100 * correct_t / total_t):.4f}')

                # Saving the best weight
                Cell_info["time"], Cell_info["epoch_past"] = nowDatetime, epoch

                if SAVE == True:
                    if network_learned:
                        valid_loss_min = batch_loss
                        Cell_info["best val.acc"], Cell_info["best epoch"], Cell_info["best val.loss"]= round(100 * correct_t / total_t, 4), epoch, round(valid_loss_min, 4)
                        # save model
                        torch.save(model.state_dict(), saving_dir + "/" + NAME + ".pt" )
                        # save Epoch in .txt
                        f = open(TXT_PATH, 'a'); f.write("Epoch " + str(epoch) + "\n") ; f.close()
                        # save Cell_info in .json
                        a_file = open(DICT_PATH_now, "w"); json.dump(Cell_info, a_file); a_file.close()
                        print('Improved - SAVE the current model')
                    else:
                        print('Not improved')
                else:
                    if network_learned:
                        valid_loss_min = batch_loss
                        Cell_info["best val.acc"], Cell_info["best epoch"], Cell_info["best val.loss"]= round(100 * correct_t / total_t, 4), epoch, round(valid_loss_min, 4)
                        print('Improved - NOT save the model, just check')
                    else:
                        print('Not improved')
        
                print(Cell_info, "\n", "-----"*15)
                #---
                model.train()
                time.sleep(0.3)
            ### end of <validation> with tqdm ...
        ### end of <validation> with torch.no_grad()
    ### end of <training> with tqdm ...
### end of <training> for epoch in range(1, n_epochs+1)
f = open(TXT_PATH, 'a')
f.write("-----\n")
f.close()

Load model to test

In [ ]:
##### LOAD the best model #####
if LOAD == True:
    if device == "cpu":
        MODEL = TypicalCNN() 
    else:
        MODEL = TypicalCNN().to(device)
    #---
    if LOAD_now == True:
        LOAD_loc = saving_dir + "/" + NAME + ".pt"
        print("nowDatetime", nowDatetime)
    else:
        LOAD_loc = result_dir + "/classify/" + TIMESTAMP + "/" + NAME + ".pt"
        print("TIMESTAMP", TIMESTAMP)
    #---
    #MODEL.load_state_dict(torch.load(LOAD_loc)) # load GPU model to GPU
    MODEL.load_state_dict(torch.load(LOAD_loc, map_location=device)) # load GPU model to CPU model
else:
    MODEL = model
#---
MODEL.eval()
print(device, "\n", LOAD_loc, "\n----------")
print(MODEL)
model_dict["TypicalCNN"] = MODEL

Test

In [ ]:
##### Test #####
label_num = 6000
label_correct = list(0. for i in range(label_num))
label_total = list(0. for i in range(label_num))
correct_t = 0
#---
time.sleep(0.5)
with torch.no_grad():
    MODEL.eval()
    with tqdm(test_loader, unit="batch") as tepoch:
        for data_t, target_t in tepoch:
            tepoch.set_description(f"Test: ")
            if device != "cpu": data_t, target_t = data_t.to(device), target_t.to(device) # on GPU
            outputs_t = MODEL(data_t)
            _, pred_t = torch.max(outputs_t, dim=1)
            c_t = (pred_t == target_t).squeeze()
            correct_t_now = (pred_t == target_t).sum().item()
            accuracy_t_now = correct_t_now / batch_size
            # correct_t += torch.sum(pred_t==target_t).item()
            correct_t += correct_t_now
            # total_t += target_t.size(0)
            for i in range(target_t.size(0)):
                trgt = target_t[i]
                label_correct[trgt] += c_t[i].item()
                label_total[trgt] += 1
            #---
            tepoch.set_postfix(batch_accuracy =100. * accuracy_t_now, position = 0) #; time.sleep(0.1)
            time.sleep(0.5)
#---                
# for i in range(batchsize):
#     print('Accuracy of %5s : %2d %%' % (classes[i], 100 * class_correct[i] / class_total[i]))
# a = sum(label_total); b = len(test_loader) * batch_size
# print("correct:", a, "/", b, "=", a/b*100)
# plt.plot(range(0, label_num), label_total)
print(correct_t, "/", sum(label_total),  "=", 100 * correct_t / sum(label_total), "%")

In [ ]:
print(correct_t, "/", sum(label_total),  "=", 100 * correct_t / sum(label_total), "%")